In [ ]:
# STEP 1

import os
import shutil
import csv

# Definisco le estensioni dei file per ciascuna tipologia
audio_est = ['.mp3', '.wav', '.flac']
docs_est = ['.txt', '.pdf', '.odt', '.docx']
images_est = ['.png', '.jpg', '.jpeg', '.gif']

# Funzione per determinare la categoria del file
def categoria_file(nome_file):
    est = os.path.splitext(nome_file)[1].lower()
    if est in audio_est:
        return 'audio'
    elif est in docs_est:
        return 'docs'
    elif est in images_est:
        return 'images'
    else:
        return None

# Percorso della cartella principale
cartella_principale = r'C:\Users\adryc\files\files'
# Cambiare la directory corrente
os.chdir(cartella_principale)

# Percorso del file di recap
recap_file = os.path.join(cartella_principale, 'recap.csv')

# Lista dei file da spostare
files = [f for f in os.listdir(cartella_principale) if os.path.isfile(os.path.join(cartella_principale, f))]
files.sort()

# Verifico l'esistenza del file recap.csv, se non esiste lo creo e imposto l'intestazione 
if not os.path.exists(recap_file):
    with open(recap_file, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Nome', 'Tipo', 'Dimensione (byte)'])

# Apro il file recap.csv in modalità append
with open(recap_file, mode='a', newline='') as file:
    writer = csv.writer(file)

      # Lista dei file da spostare, ordinati alfabeticamente
    files = sorted(f for f in os.listdir(cartella_principale) if os.path.isfile(os.path.join(cartella_principale, f)))

    # Itero sui file e li sposto nelle sottocartelle appropriate
    for nome_file in files:
        file_path = os.path.join(cartella_principale, nome_file)
        file_type = categoria_file(nome_file)
        
        if file_type:
            target_folder = os.path.join(cartella_principale, file_type)
            os.makedirs(target_folder, exist_ok=True)
            shutil.move(file_path, os.path.join(target_folder, nome_file))
            
            # Ottengo informazioni sul file
            file_size = os.path.getsize(os.path.join(target_folder, nome_file))
            
            # Stampo le informazioni
            print(f"Nome: {nome_file}, Tipo: {file_type}, Dimensione: {file_size} byte")
            
            # Scrivo le informazioni nel file recap.csv
            writer.writerow([nome_file, file_type, file_size])


In [ ]:
# STEP 3

import os
from PIL import Image
import numpy as np
from tabulate import tabulate

# Percorso della cartella principale
cartella_principale = r'C:\Users\adryc\files\files\images'
# Cambio la directory corrente
os.chdir(cartella_principale)

# Funzione per analizzare ogni singola immagine 
def analizza_immagine(image_path):
   
    img = Image.open(image_path) # carico un'immagine
    img_array = np.array(img) # la converto in array
    altezza, larghezza = img_array.shape[:2] # determino le dimensioni
   
    # Immagine in scala di grigio
    if img_array.ndim == 2:
        tipo = "Scala di grigio"
        media_grayscale = np.mean(img_array)
        return { # creo un dizionario
            "nome_file": os.path.basename(image_path),
            "altezza": altezza,
            "larghezza": larghezza,
            "tipo": tipo,
            "grayscale": media_grayscale,
            "R": None,
            "G": None,
            "B": None,
            "ALPHA": None
        }
    # Immagine RGB
    elif img_array.ndim == 3:
        if img_array.shape[2] == 3:
            tipo = "RGB"
            media_r = np.mean(img_array[:, :, 0])
            media_g = np.mean(img_array[:, :, 1])
            media_b = np.mean(img_array[:, :, 2])
            return {
                "nome_file": os.path.basename(image_path),
                "altezza": altezza,
                "larghezza": larghezza,
                "tipo": tipo,
                "grayscale": None,
                "R": media_r,
                "G": media_g,
                "B": media_b,
                "ALPHA": None
            }
        # Immagine RGBA
        elif img_array.shape[2] == 4:
            tipo = "RGBA"
            media_r = np.mean(img_array[:, :, 0])
            media_g = np.mean(img_array[:, :, 1])
            media_b = np.mean(img_array[:, :, 2])
            media_a = np.mean(img_array[:, :, 3])
            return {
                "nome_file": os.path.basename(image_path),
                "altezza": altezza,
                "larghezza": larghezza,
                "tipo": tipo,
                "grayscale": None,
                "R": media_r,
                "G": media_g,
                "B": media_b,
                "ALPHA": media_a
            }
    else:
        return {
            "nome_file": os.path.basename(image_path),
            "altezza": altezza,
            "larghezza": larghezza,
            "tipo": "sconosciuto",
            "grayscale": None,
            "R": None,
            "G": None,
            "B": None,
            "ALPHA": None
        }

# Funzione per creare la tabella riassuntiva
def crea_tabella_riassuntiva(images_folder):
    
    dati = [] # creo una lista con i dati delle immagini
    for file_name in os.listdir(images_folder): # itero su tutti i file nella cartella
        if file_name.lower().endswith(('png', 'jpg', 'jpeg', 'bmp', 'gif')): # controllo se il file ha un'estensione che indica un'immagine. 
            file_path = os.path.join(images_folder, file_name)
            info = analizza_immagine(file_path)
            dati.append([ # aggiungo le informazioni delle immagini alla lista
                info["nome_file"],
                info["altezza"],
                info["larghezza"],
                info["tipo"],
                info["media_grayscale"],
                info["media_r"],
                info["media_g"],
                info["media_b"],
                info["media_a"]
            ])
    # imposto l'intestazione della tabella
    headers = ["Nome Immagine", "Altezza (px)", "Larghezza (px)", "Grayscale", "R", "G", "B", "ALPHA"]
    tabella = tabulate(dati, headers=headers, tablefmt="grid")
    return tabella

# Esecuzione dello script sulla sottocartella 'images'
images_folder = 'images'
tabella_riassuntiva = crea_tabella_riassuntiva(images_folder)
print(tabella_riassuntiva) # stampa la tabella
